# 编程实现RSA密码系统

In [1]:
import numpy as np
import random


def cut(obj, sec):
    # 字符串切分
    return [obj[i:i+sec] for i in range(0, len(obj), sec)]


class RSA_PublicKeyCryptosystem:
    def __init__(self, max_bit=14) -> None:
        self.max_bit = max_bit
        # 随机产生大素数p,q(位长默认14-bit)
        # 先确定素数范围
        max_ = 1 << self.max_bit

        self.isPrime = np.ones(max_, dtype=int)
        self.primes = []
        for i in range(2, max_, 1):  # 素数筛法
            if self.isPrime[i]:
                if i >= (1 << (self.max_bit-1)):
                    # 满足位长bit即加入选择范围
                    self.primes.append(i)
                for j in range(i+i, max_, i):
                    self.isPrime[j] = 0

        self.p, self.q = random.sample(self.primes, 2)
        self.n = self.p * self.q

        self.l = -1  # 密文信息空间需满足10^l<=n
        tmp = 1
        while True:
            if tmp > self.n:
                break
            tmp *= 10
            self.l += 1

        # 随机产生公/私钥对(e,n)及d
        self.phi = (self.p-1) * (self.q-1)
        self.e = random.randint(2, self.phi-1)
        while self.gcd(self.phi, self.e) != 1:
            # 使φ与e互素
            self.e = random.randint(2, self.phi-1)

        _, self.d = self.ex_gcd(self.phi, self.e)
        if self.d < 0:
            self.d += self.phi  # 保证d为正

    def Encryption(self, str_):  # 加密，可以满足汉字
        # 对消息进行数字化
        str_utf8 = str_.encode("UTF-8")  # UTF-8编码
        str_hex = str_utf8.hex()
        str_digit = str(int(str_hex, 16))  # 转化为表示10进制的字符串
        # 切分成满足大小的m序列
        while(len(str_digit) % (self.l)):  # 前置0，方便分批处理
            str_digit = '0' + str_digit
        m_set = cut(str_digit, self.l)
        c_str = ""
        for m in m_set:
            c = self.pow(int(m), self.e)
            c_str += str(c).zfill(self.l+1)  # 大小有讲究

        c_str = hex(int(c_str))[2:]  # 去除0x

        if len(c_str) % 2:  # 位数为奇
            c_str = '0' + c_str
        str_utf8_ = bytes.fromhex(c_str)
        # 此时进行UTF-8解码容易错误，因为不能严格解码，因此返回字符节bytes
        return str_utf8_

    def Decryption(self, str_utf8):  # 解密
        # str_utf8 = str_.encode("UTF-8")
        str_hex = str_utf8.hex()
        str_digit = str(int(str_hex, 16))  # 转化为表示10进制的字符串
        # 切分成满足大小的c序列
        while(len(str_digit) % (self.l+1)):  # 前置0
            str_digit = '0' + str_digit

        c_set = cut(str_digit, self.l+1)
        m_str = ""
        for c in c_set:
            m = self.pow(int(c), self.d)
            m_str += str(m).zfill(self.l)

        m_str = hex(int(m_str))[2:]  # 去除0x

        if len(m_str) % 2:  # 位数为奇
            m_str = '0' + m_str
        str_utf8_ = bytes.fromhex(m_str)
        return str_utf8_.decode('UTF-8', 'ignore')  # UTF-8解码，忽略错误解码

    def pow(self, x, y):  # 快速幂(模重复平方法)
        ans = 1
        while(y):
            if(y & 1):
                ans *= x
                ans = ans % self.n
            y = y >> 1
            x = x*x
            x = x % self.n
        return ans

    def gcd(self, a, b):
        # 求解最大公因数
        if(b == 0):
            return a
        return self.gcd(b, a % b)

    def ex_gcd(self, a, b):  # 广义欧几里得算法
        s = [0, 1, 0]
        t = [0, 0, 1]
        r = [a, b]
        r.append(r[0] % r[1])
        q = [0, 0, r[0] // r[1]]
        # 注：空间复杂度可以改进
        n = 3
        while True:
            q.append(r[n - 2] // r[n - 1])
            r.append(r[n - 2] % r[n - 1])
            s.append(s[n - 2] - (q[n - 1] * s[n - 1]))
            t.append(t[n - 2] - (q[n - 1] * t[n - 1]))
            if r[n] == 0:
                break
            n = n + 1
        # print("s= "+str(s[n]), "t= "+str(t[n]))
        return s[n], t[n]


In [4]:
rsa = RSA_PublicKeyCryptosystem(max_bit=14)
secret = rsa.Encryption(
    "Mathematical Fundation of Information security + 201405001＋学号")
print(secret)


b'a\\\xc9\xd1\xe3^\x12\x10\xf1\xdao\x9cN^\x807\x08b\xe8\x03\x7fJ\xd7C\x0ev\xeb\xa2\x82MfAP4\xd3_\x9b\xdb< @!\x19\xd3\xbcx\x9e\x9c\xbeg\xda@\x1d\x8f\xc0\x86k\xde\x1dx\xfa\x0b\x19\xae\xc7\xe1R\xf9>\x9d%\x8f\xc9d\x9f=<:'


In [5]:
rsa.Decryption(secret)


'Mathematical Fundation of Information security + 201405001＋学号'